In [14]:
# =========================================================
# SEÇÃO 1: IMPORTAÇÕES E SETUP GERAL
# =========================================================

In [15]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import warnings
import itertools

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.datasets import get_rdataset
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS

from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [16]:
# =========================================================
# SEÇÃO 2: FUNÇÕES AUXILIARES (SETUP E PROCESSAMENTO)
# =========================================================

In [17]:
def definir_seed(seed_value=42):
    np.random.seed(seed_value)
    random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

In [18]:
def salvar_dataset(serie, dataset_name):
    dir_path = "./datasets"
    os.makedirs(dir_path, exist_ok=True)
    file_path = os.path.join(dir_path, f"{dataset_name.lower()}.csv")
    df = pd.DataFrame({"date": serie.index, "value": serie.values})
    df.to_csv(file_path, index=False)
    print(f"-> Cópia do dataset '{dataset_name}' salva em: {file_path}")

def carregar_serie(nome):
    print(f"Buscando dados de '{nome}' via statsmodels...")
    nome_base = nome.lower()

    if nome_base == "airpassengers":
        df = get_rdataset("AirPassengers", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1949-01-01", periods=len(df), freq="MS"),
                          name="AirPassengers")
    elif nome_base == "lynx":
        df = get_rdataset("lynx", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1821", periods=len(df), freq="A"), name="Lynx")
    elif nome_base == "co2":
        df = get_rdataset("CO2", package="datasets").data
        df = df.ffill()
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1958-03-29", periods=len(df), freq="MS"),
                          name="CO2")
    elif nome_base == "sunspots":
        df = get_rdataset("sunspots", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1749-01-01", periods=len(df), freq="MS"),
                          name="Sunspots")
    elif nome_base == "austres":
        df = get_rdataset("austres", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1971-03-01", periods=len(df), freq="QS-MAR"),
                          name="AustralianResidents")
    elif nome_base == "nottem":
        df = get_rdataset("nottem", package="datasets").data
        serie = pd.Series(df['value'].values, index=pd.date_range(start="1920-01-01", periods=len(df), freq="MS"),
                          name="Nottingham")
    else:
        raise ValueError(f"Série '{nome}' não reconhecida.")

    salvar_dataset(serie, nome)
    return serie

In [19]:
def dividir_serie_temporal(serie, percentual_treino=0.7, percentual_validacao=0.15):
    # ... (código da função inalterado) ...
    tamanho_total = len(serie)
    if tamanho_total < 20: 
        percentual_treino=0.8
        percentual_validacao=0.0
    ponto_corte_treino = int(tamanho_total * percentual_treino)
    ponto_corte_validacao = int(tamanho_total * (percentual_treino + percentual_validacao))
    treino = serie.iloc[:ponto_corte_treino]
    validacao = serie.iloc[ponto_corte_treino:ponto_corte_validacao]
    teste = serie.iloc[ponto_corte_validacao:]
    return treino, validacao, teste

def preparar_dados_para_neuralforecast(serie, nome_serie):
    df = serie.reset_index()
    df.columns = ['ds', 'y']
    df['unique_id'] = nome_serie
    return df

In [20]:
def calculate_mape(y_true, y_pred):
    """Calcula o Mean Absolute Percentage Error (MAPE)."""
    epsilon = 1e-10
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

def calculate_mase(y_true, y_pred, y_train):
    """Calcula o Mean Absolute Scaled Error (MASE)."""
    n = len(y_train)
    if n <= 1: return np.nan
    d = np.sum(np.abs(y_train[1:] - y_train[:-1])) / (n - 1)
    if d == 0: return np.inf
    errors = np.mean(np.abs(y_true - y_pred))
    return errors / d

def calcular_metricas(y_true, y_pred, y_train):
    """Calcula um dicionário com todas as métricas de erro."""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = calculate_mape(y_true, y_pred)
    mase = calculate_mase(y_true, y_pred, y_train)
    return {'RMSE': rmse, 'MAPE': mape, 'MASE': mase}

In [21]:
# =========================================================
# SEÇÃO 3: NOVAS FUNÇÕES PARA O MODELO HÍBRIDO HYS-MF
# =========================================================

In [22]:
def preprocess_residuals_for_direct_forecast(residuals, h, input_size):
    """
    Prepara os dados de resíduos para a abordagem de previsão direta.
    Para cada horizonte h, cria um target específico.
    """
    X, y = [], []
    for i in range(len(residuals) - input_size - h + 1):
        X.append(residuals[i : i + input_size])
        y.append(residuals[i + input_size : i + input_size + h])
    
    X, y = np.array(X), np.array(y)
    
    # Gera H dataframes, um para cada horizonte
    dfs_por_horizonte = []
    for i in range(h):
        df = pd.DataFrame()
        df['y'] = y[:, i] # Target é o resíduo do i-ésimo passo a frente
        for j in range(input_size):
            df[f'lag_{j+1}'] = X[:, input_size - 1 - j]
        df['unique_id'] = 'residuos'
        dfs_por_horizonte.append(df)
        
    return dfs_por_horizonte

def treinar_e_prever_hys_mf(treino, validacao, teste, ordem_arima, seed, freq, input_size, h, max_steps=100):
    """
    Implementa a metodologia HyS-MF do artigo.
    """
    # 1. Treina o ARIMA e faz a previsão recursiva
    treino_validacao_arima = pd.concat([treino, validacao]).asfreq(freq)
    modelo_arima = ARIMA(treino_validacao_arima, order=ordem_arima).fit()
    preds_arima = modelo_arima.predict(start=teste.index[0], end=teste.index[-1])
    
    # 2. Obtém os resíduos do ARIMA no conjunto de treino+validação
    residuos = modelo_arima.resid
    
    # 3. Prepara os dados dos resíduos para a abordagem direta
    # Para o N-BEATS, passamos o formato 'ds', 'y'
    df_residuos_nf = preparar_dados_para_neuralforecast(residuos, "residuos")

    # 4. Loop para treinar um modelo N-BEATS para cada horizonte
    preds_residuos = []
    for i in range(h):
        # O N-BEATS da NeuralForecast já lida com lags, então podemos simplificar
        # Treinamos um modelo para prever h passos e pegamos apenas o i-ésimo
        modelos_residuos = [NBEATS(input_size=input_size, h=h, max_steps=max_steps, scaler_type='standard', random_seed=seed)]
        nf_residuos = NeuralForecast(models=modelos_residuos, freq=freq)
        nf_residuos.fit(df=df_residuos_nf)
        
        # Faz a previsão de h passos e seleciona apenas o que nos interessa
        pred_h_passos = nf_residuos.predict()['NBEATS'].values
        preds_residuos.append(pred_h_passos[i])

    preds_residuos = np.array(preds_residuos)
    
    # 5. Previsão Híbrida = Previsão ARIMA + Previsão dos Resíduos
    preds_hibrido = preds_arima.values + preds_residuos
    
    return preds_arima, preds_hibrido

In [23]:
# =========================================================
# SEÇÃO 4: EXECUÇÃO DO EXPERIMENTO COMPLETO
# =========================================================

In [24]:
def executar_experimento(nome_da_serie):
    try:
        N_CICLOS = 3 # Reduzido para um teste rápido devido à alta complexidade
        ORDEM_ARIMA = (3, 1, 2)
        SEED_INICIAL = 42

        serie_completa = carregar_serie(nome_da_serie)
        treino, validacao, teste = dividir_serie_temporal(serie_completa)
        
        if len(teste) < 2: return None
        freq = serie_completa.index.freqstr or pd.infer_freq(serie_completa.index)
        if freq is None: return None
        
        df_treino_validacao_nf = pd.concat([preparar_dados_para_neuralforecast(treino, nome_da_serie), 
                                            preparar_dados_para_neuralforecast(validacao, nome_da_serie)])
        
        h = len(teste)
        input_size = 2 * h
        resultados_finais = []

        for i in tqdm(range(N_CICLOS), desc=f"Ciclos para {nome_da_serie}"):
            definir_seed(SEED_INICIAL + i)
            try:
                # Modelo Híbrido (HyS-MF) e ARIMA
                preds_arima, preds_hibrido = treinar_e_prever_hys_mf(treino, validacao, teste, ORDEM_ARIMA, SEED_INICIAL + i, freq, input_size, h, max_steps=50)
                metricas_hibrido = calcular_metricas(teste.values, preds_hibrido, treino.values)
                metricas_hibrido['modelo'] = 'Híbrido (HyS-MF)'
                resultados_finais.append(metricas_hibrido)
                
                metricas_arima = calcular_metricas(teste.values, preds_arima.values, treino.values)
                metricas_arima['modelo'] = 'ARIMA'
                resultados_finais.append(metricas_arima)

                # Modelo N-BEATS (na série original)
                modelos_nbeats = [NBEATS(input_size=input_size, h=h, max_steps=50, scaler_type='standard', random_seed=SEED_INICIAL + i)]
                nf_nbeats = NeuralForecast(models=modelos_nbeats, freq=freq)
                nf_nbeats.fit(df=df_treino_validacao_nf)
                preds_nbeats = nf_nbeats.predict()['NBEATS'].values
                metricas_nbeats = calcular_metricas(teste.values, preds_nbeats, treino.values)
                metricas_nbeats['modelo'] = 'N-BEATS'
                resultados_finais.append(metricas_nbeats)
            except Exception as e:
                print(f"AVISO: Ciclo {i+1} falhou. Erro: {e}")
                continue

        if not resultados_finais: return None
        
        df_resultados = pd.DataFrame(resultados_finais)
        df_sumario = df_resultados.groupby('modelo').agg(['mean', 'std'])
        return df_sumario.reindex(columns=['RMSE', 'MAPE', 'MASE'], level=0)
        
    except Exception as e:
        print(f"ERRO GERAL no dataset '{nome_da_serie}': {e}")
        return None


In [25]:
# =========================================================
# SEÇÃO 5: ORQUESTRADOR E RELATÓRIO FINAL
# =========================================================

In [ ]:
LISTA_DE_DATASETS = ['AirPassengers'] # Começando com um para teste
resultados_gerais = {}

for dataset in LISTA_DE_DATASETS:
    sumario = executar_experimento(dataset)
    if sumario is not None:
        resultados_gerais[dataset] = sumario

print("\n\n" + "="*60)
print("     RELATÓRIO FINAL: COMPARAÇÃO DE MODELOS")
print("="*60)

if resultados_gerais:
    # Exibindo o resultado para o primeiro dataset da lista como exemplo
    primeiro_dataset = LISTA_DE_DATASETS[0]
    df_sumario_exemplo = resultados_gerais.get(primeiro_dataset)
    if df_sumario_exemplo is not None:
        print(f"Resultados para o dataset: {primeiro_dataset}")
        # Reutilizando a função de exibição que já tínhamos
        # exibir_sumario_estilizado(df_sumario_exemplo) # Se tiver essa função definida
        display(df_sumario_exemplo.style.format('{:.3f}'))
else:
    print("Nenhum experimento foi concluído com sucesso.")

Buscando dados de 'AirPassengers' via statsmodels...
-> Cópia do dataset 'AirPassengers' salva em: ./datasets\airpassengers.csv


Ciclos para AirPassengers:   0%|          | 0/3 [00:00<?, ?it/s]Seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.5 M  | train
-------------------------------------------------------
2.5 M     Trainable params
3.0 K     Non-trainable params
2.5 M     Total params
10.064    Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 49: 100%|██████████| 1/1 [00:00<00:00,  3.69it/s, v_num=122, train_loss_step=0.0741, train_loss_epoch=0.0741]

`Trainer.fit` stopped: `max_steps=50` reached.


Epoch 49: 100%|██████████| 1/1 [00:00<00:00,  3.54it/s, v_num=122, train_loss_step=0.0741, train_loss_epoch=0.0741]

...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 100.57it/s]

Seed set to 42


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.5 M  | train
-------------------------------------------------------
2.5 M     Trainable params
3.0 K     Non-trainable params
2.5 M     Total params
10.064    Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 18:   0%|          | 0/1 [00:00<?, ?it/s, v_num=130, train_loss_step=0.266, train_loss_epoch=0.266]        